# Concept map connected components sizes

In [ ]:
from notebook_prelude import *

In [ ]:
data = collections.defaultdict(list)
graphs = {}
for dataset in helper.log_progress(dataset_helper.get_dataset_names_with_concept_map()):
    X, Y = dataset_helper.get_concept_map_for_dataset(dataset)
    X = graph_helper.get_graphs_only(X)
    graphs[dataset] = X
    connected_components = [list(nx.connected_components(nx.Graph(x))) for x in X]
    data['dataset'].append(dataset)
    data['connected_components'].append(connected_components)
df = pd.DataFrame(data).set_index('dataset')

In [ ]:
df['size_connected_components'] = df.connected_components.apply(lambda x: [len(y) for y in x])
df['sizes_connected_component'] = df.connected_components.apply(lambda x: [[len(y_) for y_ in y] for y in x])
df['num_connected_components'] = df.size_connected_components.apply(lambda x: sum(x))

In [ ]:
df.loc['ling-spam'].sizes_connected_component

In [ ]:
def plot_connected_component_size(df_, dataset, ax=None, **plot_args):
    if ax is None:
        _, ax = plt.subplots(figsize=(11, 5))
    fig = ax.get_figure()
    s = pd.Series(df_.size_connected_components)
    s.plot(kind='hist', ax = ax, label=dataset, normed=True, title='Dataset: {}'.format(dataset), **plot_args)
    ax.set_xlabel('# of nodes per connected component')
    fig.tight_layout()
    return ax

for dataset, df_ in df.iterrows():
    ax = plot_connected_component_size(df_, dataset, alpha=0.8)
    fig = ax.get_figure()
    save_fig(fig, 'connected_component_size_{}'.format(dataset), 'tmp/connected_component_size')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
LIMIT = ['ng20', 'ted_talks']
for dataset, df_ in df.iterrows():
    if dataset not in LIMIT: continue
    ax = plot_connected_component_size(df_, dataset, bins=30, alpha=0.8, ax = ax)
    
    ax.set_title('')
ax.grid(False)
ax.legend()
fig.tight_layout()
save_fig(fig, 'connected_component_size_comparison', 'tmp/connected_component_size')

## Percentage of graphs with more than one connected component

In [ ]:
#df.loc['ling-spam']
#df.connected_components
#df['num_more_than_one_connected_component'] = df.size_connected_components.apply(lambda x: x >)
df['ratio_more_than_one'] = df.apply(lambda x: len([y for y in x.size_connected_components if y > 1]) / len(x.size_connected_components),axis=1)
df_ = df[['ratio_more_than_one']] * 100
df_.loc['mean'] = df_.mean()
print((df_).to_latex(float_format = '%.1f'))

In [ ]:
def get_ratio_of_connected_component_size_for_dataset(x, connected_component_size=2):
    c_ = 0
    s_ = 0
    for cc in x.sizes_connected_component:
        elements = [c for c in cc if c == connected_component_size]
        c_ += len(elements)
        s_ += sum(elements)
    return c_ / x.num_connected_components

for i in range(2, 5):
    df['ratio_connected_component_of_size_' + str(i)] = df.apply(lambda x: get_ratio_of_connected_component_size_for_dataset(x, i), axis=1)

ratio_columns = sorted([c for c in df.columns if c.startswith('ratio') and not c.endswith('1')])
df_ = (df[ratio_columns] * 100).rename(columns = {c: '|s_c|={}'.format(c.split('_')[-1]) for c in ratio_columns})
df_.loc['mean'] = df_.mean()
print(df_.to_latex(float_format = '%.1f'))
df_